In [1]:
from aicsimageio import AICSImage
from tifffile import imsave
from pathlib import Path
import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import ball, binary_closing, remove_small_objects, dilation, erosion, disk
from aicssegmentation.core.pre_processing_utils import  intensity_normalization, edge_preserving_smoothing_3d
from aicssegmentation.core.vessel import vesselness3D
from aicssegmentation.core.seg_dot import dot_2d
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
import matplotlib.pyplot as plt
from itkwidgets import view   
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
plt.rcParams["figure.figsize"] = [16, 12]
from aicssegmentation.core.MO_threshold import MO
from aicspylibczi import CziFile

# dots|
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.seg_dot import dot_2d_slice_by_slice_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects, dilation, erosion, ball     # function for post-processing (size filter)
from skimage.feature import peak_local_max
from skimage.measure import label
from scipy.ndimage import distance_transform_edt
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects, dilation, erosion, ball     # function for post-processing (size filter)
from skimage.feature import peak_local_max
from skimage.measure import label
from scipy.ndimage import distance_transform_edt

# function for core algorithm
from aicssegmentation.core.vessel import filament_2d_wrapper,filament_3d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d
from aicssegmentation.core.utils import get_middle_frame, hole_filling, get_3dseed_from_mid_frame
from skimage.morphology import remove_small_objects, dilation, ball, opening, cube

In [3]:
filename = r"\\allen\aics\assay-dev\users\Sandi\nuc-morph-analysis\speckle-analysis\raw\20230425_L02-01_processed_C=0_T=1.tiff"
reader = AICSImage(filename) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

# previous: (1, 1, 148, 2047, 848)

The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


(1, 1, 148, 2047, 848)


In [4]:
#####################
structure_channel = 0

#####################

struct_img0 = IMG[0,structure_channel,:,:,:].copy()

#view(single_fluorescent_view(struct_img0))

In [5]:
struct_img0.shape

(148, 2047, 848)

<h3> Current Workflow <h3>

<h4> Speckle Segmentation Workflow <h4>

In [144]:
# pre-processing

# Intensity scaling and smoothing
intensity_scaling_param = [5000] # current: [1,18]
gaussian_smoothing_sigma = 1

# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param) # struct_img0 or pcna_original 

# smoothing with gaussian filter
structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)

intensity norm: min-max norm with upper bound 5000


In [145]:
# vesselness filter

vesselness_sigma1 = [1.5] # original:2, current:1 |increase for thicker segmentation
vesselness_cutoff1 = 0.17 # original:0.07, current:0.17 |increase for thinner/less segmentation (was at 0.34)

response1 = vesselness3D(structure_img_smooth, sigmas=vesselness_sigma1,  tau=1, whiteonblack=True)
bw1 = response1 > vesselness_cutoff1


In [146]:
# viewer_bw = view(segmentation_quick_view(bw1))
# viewer_bw

In [147]:
bw1.shape

(148, 2047, 848)

In [148]:
# bw2 = structure_img_smooth>0.5
# bw2 = np.invert(bw2)
# bw = np.logical_or(bw1, bw2)

In [149]:
# save 
# bw1 = np.invert(bw1)
seg = bw1 >0
out=seg.astype(np.uint8)
out[out>0]=255
imsave("20230425_L02-01_SEGV2vesselness_C=0_T=500.tif", out)

<h3> Processing speckle and lamin shell results <h3>

In [16]:
structure_channel = 0

# read in lamin shell segmentation OR benji's instant seg
filename = r"\\allen\aics\assay-dev\users\Sandi\nuc-morph-analysis\speckle-analysis\seg\seg_laminshell\SEGlaminshell_20230425_L02-01_processed_C=0_T=530.tiff"
reader = AICSImage(filename) 
lamin = reader.data.astype(np.uint8)
struct_lamin = lamin[0,structure_channel,:,:,:].copy()

# read in speckle segmentation (called vesselness)
filename = r"\\allen\aics\assay-dev\users\Sandi\nuc-morph-analysis\speckle-analysis\seg\seg_vesselness\SEGvesselness_20230425_L02-01_processed_C=0_T=530.tiff"
reader = AICSImage(filename) 
speckle = reader.data.astype(np.uint8)
struct_speckle = speckle[0,structure_channel,:,:,:].copy()


print("lamin:", struct_lamin.shape)
print("speckle:", struct_speckle.shape)


Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0
Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


lamin: (148, 2047, 848)
speckle: (148, 2047, 848)


In [17]:
# viewer_bw = view(segmentation_quick_view(struct_speckle))
# viewer_bw

In [18]:
# get the intersection between speckle and lamin shell segmentation
intersection = speckle & lamin
intersection.dtype

dtype('uint8')

In [19]:
# save image 
imsave("SEGintersection_T=530.tiff", intersection)

In [20]:
# subtract intersection from speckle segmentation
subtracted = speckle - intersection

In [21]:
subtracted.dtype

dtype('uint8')

In [22]:
# save image 
imsave("SEGfinalspeckle_T=530.tiff", subtracted)

<h3> Code not currently in use <h3>

In [ ]:
# 3D FILAMENT FILTER 

# f3_param = [[1, 0.15]] original: 3d filament filter [1, 0.25]
# filament = filament_3d_wrapper(structure_img_smooth, f3_param)
# filament=np.invert(filament)

# # post-processing

# minArea = 400 
# seg = remove_small_objects(filament>0, min_size=minArea, connectivity=1, in_place=False) #removed spots 
# imsave("20230425_L02-01_SEGremovesmallobj_C=0_T=247.tif", filament)

In [ ]:
# # OTSU THRESHOLDING VERSION 1

# # otsu thresholding: higher value, more segmentation
# th = 1* threshold_otsu(structure_img_smooth) # original at 1.4 # current: 3.2 
# overall_shape = dilation(
#     remove_small_objects(structure_img_smooth > th, min_size=10, connectivity=1, in_place=False), #min_size = 10
#     ball(1)
# )
# fixed_shape = erosion(
#     hole_filling(overall_shape, hole_min=0, hole_max=5, fill_2d=True), #, hole_min= 0 hole_max = 5
#     ball(1)
# )

# fixed_shape = np.invert(fixed_shape)

# imsave("20230425_L02-01_SEGotsu_C=0_T=247.tif", fixed_shape)

In [35]:
# # OTSU THRESHOLDING VERSION 2

# # different otsu attempt
# struct_img_otsu = intensity_normalization(struct_img, scaling_param=intensity_norm_param_otsu)
# structure_img_smooth_otsu = edge_preserving_smoothing_3d(struct_img_otsu)

# intensity_norm_param = [0.5, 10]
# intensity_norm_param_otsu = [0.5, 15]
# scaling_factor = 1.0
# vesselness_sigma = [1.0]
# vesselness_cutoff = 0.0125
# minArea = 20
# dot_3d_sigma = 1.0

# otsu_thresh = threshold_otsu(structure_img_smooth_otsu)
# global_thresh = otsu_thresh * scaling_factor
# bw_otsu_mask = structure_img_smooth_otsu > global_thresh

# response_f3 = vesselness3D(structure_img_smooth, sigmas=vesselness_sigma, tau=1, whiteonblack=True)
# response_f3 = response_f3 > vesselness_cutoff

# response_s3_1 = dot_3d(structure_img_smooth, log_sigma=dot_3d_sigma)
# response_s3_3 = dot_3d(structure_img_smooth, log_sigma=3)

# bw_small_inverse = remove_small_objects(response_s3_1 > 0.035, min_size=150)
# bw_small = np.logical_xor(bw_small_inverse, response_s3_1 > 0.025)
# bw_medium = np.logical_or(bw_small, response_s3_1 > 0.07)
# bw_large = np.logical_or(response_s3_3 > 0.2, response_f3 > 0.25)
# bw_dots = np.logical_or(np.logical_or(bw_small, bw_medium), bw_large)
# bw = np.logical_and(bw_dots, bw_otsu_mask)

In [36]:
# # OTSU THRESHOLDING VERSION 3

# # otsu mask
# scaling_factor = 1.2

# otsu_thresh = threshold_otsu(structure_img_smooth)
# global_thresh = otsu_thresh * scaling_factor
# bw_otsu_mask = structure_img_smooth > global_thresh

In [66]:
# # 2D SPOT FILTER VERSION 1

# s2 = np.zeros_like(filament)
# for z in range(structure_img_smooth.shape[0]):
#     zslice = dot_2d(structure_img_smooth[z,], 2)
#     s2[z, :, :] = zslice < 0.07 #original: > 0.03 current  
# spot_filter = np.logical_or(s2, filament)

# imsave("20230425_L02-01_SEG_spot2d_C=0_T=247.tif", spot_filter)

In [ ]:
# # 2D SPOT FILTER VERSION 2

# s2_param = [[3, 0.01]] # current: [[2, 0.01]]
# bw_spot2d = dot_2d_slice_by_slice_wrapper(structure_img_smooth, s2_param)
# bw_spot2d_inv = np.invert(bw_spot2d)

In [75]:
# # SUBTRACTION OF TWO DIFFERENT METHODS

# subtracted = filament.astype(np.uint8) - spot_filter.astype(np.uint8)
# imsave("20230425_L02-01_SEG_subtracted_C=0_T=247.tif", subtracted)

In [70]:
# # 2D EROSION THINNING

# for z in range(bw1.shape[0]):
#     zslice= bw1[z,]
#     zslice = erosion(zslice, disk(1))
#     bw[z, :, :] = zslice

# response_vessel = vesselness3D(structure_img_smooth, sigmas=[1], tau=1, whiteonblack=True)

# seg = np.logical_or(response_vessel > 2.5, bw1) # response_vessel > 0.28 increasing parameter thins out lines

# seg = hole_filling(seg, hole_min=0, hole_max=15, fill_2d=True)# hole_min=0 # hole_max=15

# seg = np.invert(seg)
# imsave("20230425_L02-01_SEGerosionvessel_C=0_T=247.tif", seg)

In [22]:
# MASK OBJECT THRESHOLDING

# bw2, object_for_debug = MO(seg, global_thresh_method='tri', object_minArea=1200, return_object=True)
# # # Mask Object Thresholding (note: try this for speckle segmentation)
# mo_thresh, object_for_debug = MO(structure_img_smooth, global_thresh_method='tri', object_minArea=1200, return_object=True, 
#                           local_adjust=1.75) # exp: 1.95, v2 v3: 2.75

In [23]:
# # THINNING

# thin_dist_preserve=0.09
# thin_dist=2
# bw_thin = topology_preserving_thinning(bw_otsu_mask>0, thin_dist_preserve, thin_dist)